<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-Agent Customer Service System - A2A Demo**

This notebook demonstrates agent-to-agent (A2A) communication with three specialized agents coordinating to handle customer service queries.

## **System Components**
- **Customer Data Agent**: Interfaces with MCP server for customer/ticket operations
- **Support Agent**: Handles customer support questions and escalations
- **Router Orchestrator**: Coordinates between agents based on query intent

## **Prerequisites**
1. MCP server running (in separate Colab or local)
2. Google API key in Colab Secrets (as 'a5-key')
3. MCP_SERVER_URL in Colab Secrets (ngrok or localhost)

## **Clone Repository and Setup**

In [17]:
!git clone https://github.com/bhstoller/multi-agent-customer-service.git

fatal: destination path 'multi-agent-customer-service' already exists and is not an empty directory.


In [18]:
!pip install -q -r multi-agent-customer-service/requirements.txt

In [19]:
# Add repo to path
import sys
sys.path.insert(0, '/content/multi-agent-customer-service/src')

## **Configure Notebook**

In [20]:
print("Configuring notebook...")
# Import configuration libraries
from config import LOG_LEVEL, LLM_MODEL, CUSTOMER_DATA_URL, SUPPORT_URL
from google.colab import userdata
from pathlib import Path
import logging
import os
import sys

# Set Google Cloud configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'
os.environ['GOOGLE_API_KEY'] = userdata.get('a5-key')
print("- Google API Key loaded")

# Set the LLM model
print(f"- LLM Model loaded: {LLM_MODEL}")

# Set MCP Server configuration
os.environ['MCP_SERVER_URL'] = userdata.get('MCP_SERVER_URL')
print(f"- MCP Server URL loaded: {os.environ['MCP_SERVER_URL']}")

# Reload config so it picks up the new MCP Server
import importlib
import config
importlib.reload(config)

# Set up logging
log_level = getattr(logging, LOG_LEVEL)
logging.basicConfig(level=log_level, force=True)
logging.getLogger('asyncio').setLevel(log_level)
logging.getLogger('aiohttp').setLevel(log_level)
print(f"- Logging level loaded: {LOG_LEVEL}")

print("Configuration complete")

Configuring notebook...
- Google API Key loaded
- LLM Model loaded: gemini-2.0-flash
- MCP Server URL loaded: https://polar-nonsolubly-madden.ngrok-free.dev/mcp
- Logging level loaded: CRITICAL
Configuration complete


## **Import Dependencies**

In [21]:
# Handle A2A SDK import issue
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver

patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module

In [22]:
print("Importing libraries...")
# Import all modular code
# from config import MCP_SERVER_URL, CUSTOMER_DATA_URL, SUPPORT_URL, LLM_MODEL, LOG_LEVEL
from agents import customer_data_agent, customer_data_agent_card, support_agent, support_agent_card
from router import RouterOrchestrator, A2ASimpleClient
from mcp_server import create_app

# Import A2A infrastructure libraries
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor, A2aAgentExecutorConfig
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
import uvicorn
import nest_asyncio

# Import general python libraries
import asyncio
import threading
import time

# Turn off warnings for readability
import warnings
warnings.filterwarnings('ignore')

print("All imports loaded")

Importing libraries...
All imports loaded


## **Start MCP Server**

In [23]:
print("Starting the MCP Server...")
MCP_SERVER_HOST = "127.0.0.1"
MCP_SERVER_PORT = "10050"

def run_mcp_server_in_background():
    app = create_app()
    app.run(
        host= MCP_SERVER_HOST,
        port= MCP_SERVER_PORT,
        debug= False,
        use_reloader= False
    )

mcp_thread = threading.Thread(
    target= run_mcp_server_in_background,
    daemon= True
)
mcp_thread.start()
time.sleep(2)
print(f'MCP server started on http://{MCP_SERVER_HOST}:{MCP_SERVER_PORT}')

Starting the MCP Server...
 * Serving Flask app 'mcp_server'
 * Debug mode: off


Address already in use
Port 10050 is in use by another program. Either identify and stop that program, or start the server with a different port.


MCP server started on http://127.0.0.1:10050


## **Start Agent Servers**

Start Customer Data and Support agents on ports 10020 and 10021

In [24]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent."""
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [25]:
print("Starting agent servers...")
async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start Customer Data and Support agent servers."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
    ]

    await asyncio.sleep(2)
    print('Agent servers started:')
    print('   - Customer Data Agent: http://127.0.0.1:10020')
    print('   - Support Agent:       http://127.0.0.1:10021')

    try:
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')


def run_servers_in_background() -> None:
    """Run servers in a background thread."""
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# Apply nest_asyncio before starting servers
nest_asyncio.apply()

# Start servers in background thread
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()
time.sleep(3)
print("Agent servers initialized")

Starting agent servers...


ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 10020): [errno 98] address already in use
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 10021): [errno 98] address already in use


Agent servers initialized


## **Initialize The Router (Orchestrator) Agent**

In [26]:
# Create A2A client and router
a2a_client = A2ASimpleClient()
router = RouterOrchestrator(a2a_client)

print("Router Orchestrator initialized and ready for queries")

Router Orchestrator initialized and ready for queries


## **Test Scenarios**

Run test scenarios demonstrating A2A coordination:

### Scenario 1: Simple Query (Single Agent)

**Query:** Get customer information for a specific ID

**Expected Flow:**
1. Router analyzes query
2. Router calls Customer Data Agent
3. Returns customer details

In [27]:
query_1 = "Get customer information for customer ID 5"

result_1 = await router.process_query(query_1)
print(f"\nFINAL RESPONSE:\n{result_1}")


 USER QUERY: Get customer information for customer ID 5 

[ROUTER STEP 1]: The user is asking for information about a specific customer. This task falls under the capabilities of the 'customer_data' agent.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 5 ```json {   "customer": {     "id": 5,     "name": "Charlie Brown",     "email": "charlie.brown@email.com",     "phone": "+1-555-0105",     "status...

[ROUTER STEP 2]: I have successfully retrieved the customer information from the 'customer_data' agent. Now I will formulate the final answer for the user.

[ROUTER]: Task Complete.

FINAL RESPONSE:
Customer information for customer ID 5:
Name: Charlie Brown
Email: charlie.brown@email.com
Phone: +1-555-0105
Status: active


### Scenario 2: Negotiation/Escalation (Multiple Agents)

**Query:** Customer wants to cancel subscription but has billing issues

**Expected Flow:**
1. Router detects multiple intents
2. Router calls Support Agent (for cancellation policy)
3. Router calls Customer Data Agent (for billing history)
4. Router synthesizes coordinated response

In [32]:
query_2 = "I want to cancel my subscription but I'm having billing issues. My customer ID is 1."

result_2 = await router.process_query(query_2)
print(f"\nFINAL RESPONSE:\n{result_2}")


 USER QUERY: I want to cancel my subscription but I'm having billing issues. My customer ID is 1. 

[ROUTER STEP 1]: The user wants to cancel their subscription and is also having billing issues. This requires input from both agents. I will first ask the support agent for advice on cancelling a subscription. Then, I will check the customer data to create a ticket regarding the billing issues.
    >>> [A2A CALL] Connecting to support_agent at http://localhost:10021...
   <<< [A2A RESPONSE]: [SUPPORT_AGENT]: This is a data retrieval request, not a support question. PASS_TO_CUSTOMER_DATA_AGENT ...

[ROUTER STEP 2]: The support agent indicated that cancelling a subscription is a customer data issue. I will now call the customer data agent for help with cancelling the subscription, as well as create a ticket for the billing issues.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: I cannot cancel a subscription. However, I can create a ticke

### Scenario 3: Multi-Step Coordination (Complex Query)

**Query:** Show all active customers who have closed tickets

**Expected Flow:**
1. Router decomposes query into sub-tasks
2. Router calls Customer Data Agent to get active customers
3. Router calls Customer Data Agent again for ticket history
4. Router filters and synthesizes final report

In [34]:
query_3 = "Show me the names of all active customers who have closed tickets."

result_3 = await router.process_query(query_3)
print(f"\nFINAL RESPONSE:\n{result_3}")


 USER QUERY: Show me the names of all active customers who have closed tickets. 

[ROUTER STEP 1]: The user is asking for a list of active customers who have closed tickets. I need to first get a list of all active customers, then get the ticket history for each of those customers, and then filter the list to only include customers with closed tickets. I will use the customer_data agent to get the customer list and ticket history.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling list_customers with status active ```json {"customer": [{"id": 4, "name": "Alice Williams", "email": "alice.w@techcorp.com", "phone": "+1-555-0104", "status": "active", ...

[ROUTER STEP 2]: Now that I have the list of active customers, I need to get the ticket history for each of them. I will batch the request by providing all customer IDs at once to the customer_data agent.
    >>> [A2A CALL] Connecting to customer_data at http:

### Scenario 4: Simple Query

**Query:** Get customer information for ID 5

**Expected Flow:** Single agent, straightforward MCP call

In [36]:
query_4 = "Get customer information for ID 5"

result_4 = await router.process_query(query_4)
print(f"\nFINAL RESPONSE:\n{result_4}")


 USER QUERY: Get customer information for ID 5 

[ROUTER STEP 1]: The user is asking for specific customer information. I should use the customer_data agent to retrieve this information.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 5 ```json {   "customer": {     "id": 5,     "name": "Charlie Brown",     "email": "charlie.brown@email.com",     "phone": "+1-555-0105",     "status...

[ROUTER STEP 2]: I have successfully retrieved the customer information. I can now provide the final answer to the user.

[ROUTER]: Task Complete.

FINAL RESPONSE:
Customer information for ID 5:
Name: Charlie Brown
Email: charlie.brown@email.com
Phone: +1-555-0105
Status: active


### Scenario 5: Coordinated Query

**Query:** I'm customer 2 and need help upgrading my account

**Expected Flow:** Multiple agents coordinate: data fetch + support response

In [42]:
query_5 = "I'm customer 2 and need help upgrading my account"

result_5 = await router.process_query(query_5)
print(f"\nFINAL RESPONSE:\n{result_5}")


 USER QUERY: I'm customer 2 and need help upgrading my account 

[ROUTER STEP 1]: The user needs help upgrading their account. First, verify the user's ID and then ask support agent about upgrading the account.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 2 ```json {   "customer": {     "id": 2,     "name": "Jane Smith",     "email": "jane.smith@example.com",     "phone": "+1-555-0102",     "status": "...

[ROUTER STEP 2]: Now that I have confirmed Jane Smith's customer data, I can ask the support agent about account upgrades.
    >>> [A2A CALL] Connecting to support_agent at http://localhost:10021...
   <<< [A2A RESPONSE]: [SUPPORT_AGENT]: This is a data retrieval request, not a support question. PASS_TO_CUSTOMER_DATA_AGENT ...

[ROUTER STEP 3]: The support agent indicated that the request to upgrade an account should be directed towards the customer data agent. I will now ask the

### Scenario 6: Complex Query

**Query:** Show me all active customers who have open tickets

**Expected Flow:** Requires negotiation between data and support agents

In [44]:
query_6 = "Show me all active customers who have open tickets"

result_6 = await router.process_query(query_6)
print(f"\nFINAL RESPONSE:\n{result_6}")


 USER QUERY: Show me all active customers who have open tickets 

[ROUTER STEP 1]: The user is asking for a list of active customers with open tickets. Following the critical instruction, I should first get the list of all active customers. Then, I will request the ticket history for all of those customers in a single request to customer_data. Finally, I will filter the results to only return active customers with open tickets.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling list_customers with status active ```json {"customer": [{"id": 4, "name": "Alice Williams", "email": "alice.w@techcorp.com", "phone": "+1-555-0104", "status": "active", ...

[ROUTER STEP 2]: Now that I have the list of active customers, I need to get their ticket histories to determine which ones have open tickets. I will send a single request to customer_data to get the ticket history for all the customer IDs I received.
    >>> [A2

### Scenario 7: Escalation

**Query:** I've been charged twice, please refund immediately!

**Expected Flow:** Router must identify urgency and route appropriately

In [45]:
query_7 = "I've been charged twice, please refund immediately!"

result_7 = await router.process_query(query_7)
print(f"\nFINAL RESPONSE:\n{result_7}")


 USER QUERY: I've been charged twice, please refund immediately! 

[ROUTER STEP 1]: The user is requesting a refund due to being charged twice. I need to understand the context of the charges before I can initiate a refund. First, I'll check the customer data system to see if I can identify the user, then I will get the ticket history to see if the double charge is present there.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: Which customer id do you want to get the details for? ...

[ROUTER STEP 2]: I need the customer ID to proceed. I should ask the user for their customer ID.

[ROUTER]: Task Complete.

FINAL RESPONSE:
To investigate this double charge, I need your customer ID. Could you please provide it?


### Scenario 8: Multi-Intent

**Query:** Update my email to new@email.com and show my ticket history for customer ID 3

**Expected Flow:** Parallel task execution and coordination

In [47]:
query_8 = "Update my email to new@email.com and show my ticket history for customer ID 3"

result_8 = await router.process_query(query_8)
print(f"\nFINAL RESPONSE:\n{result_8}")


 USER QUERY: Update my email to new@email.com and show my ticket history for customer ID 3 

[ROUTER STEP 1]: The user has two requests: updating the email address and showing the ticket history. I'll use the 'customer_data' agent to fulfill both requests. First I'll update the email, then get the ticket history for customer ID 3.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling update_customer for ID 3 ```json {"customer": { "id": 3, "name": "Bob Johnson", "email": "new@email.com", "phone": "+1-555-0103", "status": "disabled", "created_at": "20...

[ROUTER STEP 2]: The email has been updated. Now I need to retrieve the ticket history for customer ID 3.
    >>> [A2A CALL] Connecting to customer_data at http://localhost:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer_history for ID 3 ```json {"get_customer_history_response": {"result": {"content": [{"text": "{\n  \"success\": tr

## **Summary**

The demo demonstrated:
- Simple queries routed to a single agent (customer data agent)
- Complex queries with coordination between both agents (customer data agent and support agent)
- Multi-step coordination with data aggregation via the customer data agent
- A2A communication
- LLM-based query routing and decision making